In [3]:
import pandas as pd
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline
#pd.set_option("display.max_colwidth", 200)
from IPython.display import display

In [4]:
conn = psycopg2.connect(host="localhost",database="mimic", user="postgres", password='postgres', port = '5432')
cursor = conn.cursor()

In [5]:
df =pd.read_csv('master_161208.csv')

In [6]:
df.columns.tolist()

['Unnamed: 0',
 'row_id',
 'icustay_id',
 'hadm_id',
 'subject_id',
 'age_at_intime',
 'gender',
 'height',
 'weight',
 'bmi',
 'ethnicity',
 'insurance',
 'ex_congestive_heart_failure',
 'ex_cardiac_aarrhythmias',
 'ex_valvular_disease',
 'ex_pulmonary_circulation',
 'ex_peripheral_vascular',
 'ex_hypertension',
 'ex_paralysis',
 'ex_other_neurological',
 'ex_chronic_pulmonary',
 'ex_diabetes_uncomplicated',
 'ex_diabetes_complicated',
 'ex_hypothyroidism',
 'ex_renal_failure',
 'ex_liver_disease',
 'ex_peptic_ulcer',
 'ex_aids',
 'ex_lymphoma',
 'ex_metastatic_cancer',
 'ex_solid_tumor',
 'ex_rheumatoid_arthritis',
 'ex_coagulopathy',
 'ex_obesity',
 'ex_weight_loss',
 'ex_fluid_electrolyte',
 'ex_blood_loss_anemia',
 'ex_deficiency_anemias',
 'ex_alcohol_abuse',
 'ex_drug_abuse',
 'ex_psychoses',
 'ex_depression',
 'apsiii',
 'apsiii_prob',
 'apsiii_creatinine_score',
 'admittime',
 'dischtime',
 'intime',
 'outtime',
 'dod',
 'survived_icustay',
 'survived_hadm',
 'ed_chartdate',
 

In [7]:
# districution of first care unit in filtered list (patients with icu-echo), we are definitely selecting out more cardiac surgery patients
df.groupby(['ea_first_careunit']).size()

ea_first_careunit
CCU        87
CSRU     1365
MICU      404
SICU      125
TSICU      70
dtype: int64

In [8]:
df.groupby(['ea_lv_systolic', 'ea_first_careunit']).size()

ea_lv_systolic  ea_first_careunit
-50.0           CCU                    2
                CSRU                   5
                MICU                   1
                TSICU                  1
-1.0            CCU                    6
                CSRU                  10
                MICU                  49
                SICU                  14
                TSICU                 11
 0.0            CCU                   42
                CSRU                 921
                MICU                 250
                SICU                  89
                TSICU                 41
 1.0            CCU                   11
                CSRU                 163
                MICU                  35
                SICU                   3
                TSICU                  2
 2.0            CCU                    8
                CSRU                  46
                MICU                  12
                SICU                   7
                TSICU  

In [9]:
sql = """
SELECT *
  FROM mimiciii.icustays
  """
icustays = pd.read_sql_query(sql, conn)

In [10]:
# distribution of first care unit in all mimic patients. 
icustays[['first_careunit']].groupby(['first_careunit']).size()

first_careunit
CCU       7726
CSRU      9312
MICU     21088
NICU      8100
SICU      8891
TSICU     6415
dtype: int64

In [11]:
# MICU vs CSRU mortality and EF
day30_mortality = df.ea_days_after_discharge_death <= 30
#sns.barplot(x="ea_lv_systolic", y="day30_mortality", hue ="ea_first_careunit", data=df)

In [ ]:
# look back at consort diagram... where should we apply MICU filter?  instead of distributive shock filter??

In [ ]:
#Look at the 221 patients with post echo and stratify EF change vs. mortality. Model change in EF as a predictor of mortality 